In [1]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     --------------------------------------- 55.1/55.1 kB 82.2 kB/s eta 0:00:00
     -------------------------------------- 58.8/58.8 kB 388.6 kB/s eta 0:00:00
     ------------------------------------ 133.4/133.4 kB 414.5 kB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 694.9 kB/s eta 0:00:00
     ---------------------------------------- 42.6/42.6 kB 1.0 MB/s eta 0:00:00
     -------------------------------------- 53.6/53.6 kB 548.9 kB/s eta 0:00:00
     -------------------------------------- 65.0/65.0 kB 589.3 kB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17411 sha256=bb104160b112cb9a22105c68dd37723e764b2b3ec57b5c11b677ed1c03f5f387
  Stored in directory: c:\users\emong\appdata\local\pip\cache\wheels\42\11\21\8a967ed422029aa7e3a5ddbe672ab693db8ee9bac9dfc26cc0
Successfully built googletrans
  Attempting un

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
conda-repo-cli 1.0.4 requires pathlib, which is not installed.
anaconda-project 0.10.2 requires ruamel-yaml, which is not installed.
udemyscraper 0.8.2 requires idna==3.2, but you have idna 2.10 which is incompatible.
udemyscraper 0.8.2 requires requests==2.26.0, but you have requests 2.28.2 which is incompatible.


In [25]:
import pandas as pd
import numpy as np
from googletrans import Translator

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, manhattan_distances, euclidean_distances

In [26]:
data = pd.read_csv("data/filipino_recipe_clean.csv")

In [27]:
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(data['ingredients_clean'])

In [28]:
translator = Translator()
text = "bangus, milkfish, garlic, oil"
text = translator.translate(text, dest='en')
text.text
#test = tf.transform([])

'bangus, milkfish, garlic, oil'

### Cosine Similarity

- cosine similarity recommendations makes more sense compared to two similarity metrics

In [32]:
test = tf.transform(["Bangus, milkfish, garlic, oil"])
test_cosine = cosine_similarity(tfidf_matrix, test)
recommend_cosine = sorted(range(len(test_cosine)), key = lambda sub: test_cosine[sub])[-5:]

In [30]:
#top = sorted(range(len(scores)), key = lambda i: scores[i])[-N:]
#Create blank dataframe to load the recommendations 
recommendations = pd.DataFrame(columns=['recipe', 'url','ingredients'])
count = 0
for i in recommend_cosine:
    recommendations.at[count, "recipe"] =  data["food"][i]
    recommendations.at[count, "ingredients"] = data["ingredients"][i]
    recommendations.at[count, "url"] = data["url"][i]
    #recommendations.at[count, "score"] = "{:.3f}".format(float(test_cosine[i]))
    count += 1
print(recommendations)

                                              recipe  \
0                                   Relyenong Bangus   
1                             Tinolang Bangus Recipe   
2  Genesee's Grilled Boneless Bangus in Pandan Le...   
3                                    Bangus Salpicao   
4               Fried Bangus Recipe (Fried Milkfish)   

                                                 url  \
0    https://www.kawalingpinoy.com/relyenong-bangus/   
1  https://panlasangpinoy.com/tinolang-bangus-rec...   
2  https://panlasangpinoy.com/shared-recipe-genes...   
3  https://panlasangpinoy.com/air-fryer-bangus-sa...   
4    https://panlasangpinoy.com/fried-bangus-recipe/   

                                         ingredients  score  
0  1 large whole bangus milkfish, juice of 1 lemo...  0.221  
1  1 large bangus milkfish, scales and innards re...  0.222  
2  1 medium sized boneless bangus milkfish, 2 fre...  0.260  
3  1 piece boneless milkfish, 4 teaspoons garlic ...  0.299  
4  1 large bonel

In [28]:
for i in recommend_cosine:
    print(data.iloc[i])
    print("---------------------------------------")

url                    https://www.kawalingpinoy.com/relyenong-bangus/
food                                                  Relyenong Bangus
course                                                            none
ingredients          1 large whole bangus milkfish, juice of 1 lemo...
instructions         Scale fish and remove innards, leaving belly i...
ingredients_clean    bangus milkfish juice lemon soy sauce onion ga...
ingredients_token    ['bangus', 'milkfish', 'juice', 'lemon', 'soy'...
Name: 2159, dtype: object
---------------------------------------
url                  https://panlasangpinoy.com/tinolang-bangus-rec...
food                                            Tinolang Bangus Recipe
course                                                            none
ingredients          1 large bangus milkfish, scales and innards re...
instructions         Heat the oil in a cooking pot.\r\nSaute the on...
ingredients_clean    bangus milkfish innards malunggay hot chayote ...
ingredients

### Manhattan Distance

In [13]:
test_manhattan = manhattan_distances(tfidf_matrix, test)
recommend_manhattan = sorted(range(len(test_manhattan)), key = lambda sub: test_manhattan[sub])[:5]

In [14]:
for i in recommend_manhattan:
    print(data.iloc[i])
    print("---------------------------------------")

url                  https://panlasangpinoy.com/filipino-food-fried...
food                                              Fried Tilapia Recipe
course                                                            none
ingredients          2 pieces tilapia cleaned and scales removed, 2...
instructions         Rub salt all over the fish including the insid...
ingredients_clean                                              tilapia
ingredients_token                                          ['tilapia']
Name: 315, dtype: object
---------------------------------------
url                      https://www.kawalingpinoy.com/tortang-talong/
food                                                    Tortang Talong
course                                                       Side Dish
ingredients          4 eggplants, 4 eggs, salt and pepper to taste....
instructions         With a knife or fork, prick eggplant. Arrange ...
ingredients_clean                                         eggplant egg
ingredients_

### Euclidean Distance

In [11]:
test_euclid = euclidean_distances(tfidf_matrix, test)
recommend_euclid = sorted(range(len(test_euclid)), key = lambda sub: test_euclid[sub])[:5]

In [12]:
for i in recommend_euclid:
    print(data.iloc[i])
    print("---------------------------------------")

url                  https://panlasangpinoy.com/tochong-bangus-recipe/
food                                                    Tochong Bangus
course                                                     Main Course
ingredients          1 bangus cleaned and sliced diagonally into pi...
instructions         Rub salt all over the fish. Let it stay for 10...
ingredients_clean    bangus tofu tahure ferment bean curd mash taus...
ingredients_token    ['bangus', 'tofu', 'tahure', 'ferment', 'bean'...
Name: 71, dtype: object
---------------------------------------
url                      https://www.kawalingpinoy.com/totsong-bangus/
food                                                    Totsong Bangus
course                                                            none
ingredients          1 whole bangus, scaled, gutted and cut into se...
instructions         Rinse fish and drain well. Season with salt an...
ingredients_clean    bangus canola tofu onion garlic ginger rom tom...
ingredients_t